# RAGによる文献参照および社内Q&A練習用

内容はChatGPTのAPIキーを使って、RAGを構築した。
PDFの中身を文章を数値変換（ベクトル化、Eembedding）し、LLMの検索先としてデータベースをもつようにした。
参考はYoutube

In [1]:
!pip install langchain==0.0.125 openai==0.27.2 chromadb==0.3.14 pypdf==3.7.0 tiktoken==0.3.3 gradio==3.23

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.me

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
import openai
import os

os.environ["OPENAI_API_KEY"] = "***"

In [3]:
os.listdir(".")

['.config', 'コロナ禍の保健医療対策に使われた税金はいくらなのか.pdf', 'sample_data']

In [4]:
# ファイルをアップロード

In [5]:
loader = PyPDFLoader('コロナ禍の保健医療対策に使われた税金はいくらなのか.pdf')
documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())

In [7]:
# プロンプトの定義
from langchain import PromptTemplate

template = """
あなたは親切なアシスタントです。下記の質問に日本語で回答してください。
質問：{question}
回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)

In [8]:
import gradio as gr

def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):
    query = history[-1][0]
    query = prompt.format(question=query)
    answer = qa.run(query)
    source = qa._get_docs(query)[0]
    source_sentence = source.page_content
    answer_source = source_sentence +"\n"+"source:"+source.metadata["source"] + ", page:" + str(source.metadata["page"])
    history[-1][1] = answer # + "\n\n情報ソースは以下です：\n" + answer_source
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot").style(height=400)

    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter",
            ).style(container=False)

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

下記がPDFの内容になっている。

コロナ禍の保健医療対策に
使われた税金はいくらなのか？
医療に関するマクロ費用統計は，複数公表されているが，
いずれを使ってもコロナ対策に使われた公費（税金）の全体
像を把握することは難しい。政府の資料や報道などで使用さ
れることの多い「国民医療費」，「社会保障費用統計」，「保健
医療支出（Health Expenditure）」それぞれについて，考察し
てみよう。
まず，「国民医療費」は全く使えない。なぜなら，専らレセ
プト（保険給付費）の集計に過ぎないためである。病院・診
療所への補助金も，予防接種の支出も，コロナ禍で業務が逼
迫した保健所にかかった費用も含まれていない。「国民医療費」
の公費は，生活保護法や障害者総合支援法に基づく医療給付，
地方公共団体単独実施などであり，コロナ禍以前と同様3.1 ～
3.2兆円で推移している。また，「国民医療費」は速報性にも
欠ける。執筆時点で最新のデータは2019年度だ。
次に，「社会保障費用統計」は，「国民医療費」よりも公費
の把握には有用だ。国の基幹統計でもある。しかし，2020年
度の統計は，2022年８月公表であり，「国民医療費」より早い
とは言え，もう少し早い公表があればと惜しまれる（費用統
計の公表が２年後となる理由は，決算ベースのため，国の決
算書をはじめ各制度の決算が出揃ってから集計することによ
る。決算データの提供が早まらない限り，早期化は難しい）。
2020年度の「社会保障費用統計」によると，「社会保障給
付費」の「医療」は42.7兆円で，2019年度より２兆円増えた。
「社会保障給付費」の「医療」には，「医療保険」，「公衆衛生」，
「生活保護のうち医療扶助」等が含まれている。「患者等負担
分」は含まれない。
「公衆衛生」は，公費負担医療や医療機関への補助金等を
含んでおり，「新型コロナウイルス感染症緊急包括支援交付金
一橋大学国際・公共政策大学院教授
公益財団法人医療科学研究所理事
井伊　雅子
469-472 巻頭言.indd 469 2023/01/05 9:57:00
470
などの新型コロナウイルス感染症対策に係る支出」はここに
含まれている。「ワクチン購入・流通費用（2020年度接種済み
相当分）」として2,322億円も「公衆衛生」に計上されている。
2020年度は「医療保険」の給付費が減り，「公衆衛生」の給
付費が増えたために，「社会保障給付費」の「医療」は２兆円
増えた。
「社会保障給付費」には，「医療」のほか，「年金」と「福
祉その他」の部門があり，「福祉その他」が，2019年度より
６兆円増えて33.9兆円である。その大きな理由は，雇用調整
助成金が増加したことだ。介護保険など介護対策（11.4兆円）
は，「福祉その他」に含まれている。
さらに，日本政府がOECDに提出している「保健医療支出
（Health Expenditure）」がある。その範囲には，「国民医療費」
にとどまらず，予防，介護，市販薬，保健所運営の支出など
が含まれており，国際比較にも使われる重要な統計だ。経済
統計として極めて機能的に設計されている。支出項目別，財
源別（公費，保険料，自己負担），提供主体別に公表しており，
公費分もしっかり把握できる。「国民医療費」「社会保障費用
統計」を包括した統計である。
残念ながら，日本の「保健医療支出」の推計精度は低く，
速報性にも欠ける。例えば，2020年度の「保健医療支出」の
合計額は59.9兆円であり，2019年61.2兆円より1.7兆円減少して
いるが，公表されているのは合計額のみで，詳細な項目別（治
療（入院・外来），リハビリ，介護，補助的サービス，医療財，
予防，管理）の支出は，2019年度までしか公表されていない（執
筆時）。ちなみに韓国は2021年度まで全ての項目別の支出を報
告している。
西沢（2022）によると，2009年の新型インフルエンザの世
界的大流行に備えて，ワクチン確保のために一般会計から支
出された2,082億円は，「保健医療支出」に計上されていない。
また，保健所運営をはじめ地方自治体が担っている公衆衛生
関連支出（地方費）に漏れが多い。こうしたことが改善され
ない限り，2020年度の項目別支出が公表されたとしても，コ
ロナ対策費の分析を行うには不十分である。
医療や福祉政策には国民は大きな関心がある。しかし，上
記に見たようにコロナ対策に関する大規模な支出でさえ，全
体像を理解することは経済学者にとっても一苦労だ。国民が
469-472 巻頭言.indd 470 2023/01/05 9:57:00
471
主体的な意思決定をするためにも分かりやすい統計のあり方
は不可欠だ。受給と負担に関して納得感がないところで，更
なる負担を求めるのは難しいだろう。
参考文献
西沢和彦（2022）「健康支出（Health expenditure）における予防支出推計
の改善に向けて－「社会保障施策に要する経費」を用いた再推計」『JRI
レビュー』5（100）
＊西沢論文の「健康支出」と文中の「保健医療支出」は，同じものを指し
ている。